Skeleton code for resolution theorem prover for propositional logic

Yoonsuck Choe  
20 October 2021

clause representation:  
```
[<positive-proposition-list>, <negative-proposition-list>]
```

proposition-list representation:  
```
[ 'p', 'q', 'r' ]
```

Note: `p`, `q`, `r`, `s` are propositions : CASE SENSITIVE!

Example clause:  
`[['p', 'q'] ['r']]` : This is clause P &#x2228; Q &#x2228; &#x00ac;R

In [1]:
DEBUG = True

In [2]:
def rm_item(lst, item):
    '''
    function: rm_pred()

    remove item from list
    - return [] if empty, not None
    '''
    lst.remove(item)

    if lst == None:
        return []
    else:
        return lst

In [3]:
def mk_unique(clause):
    '''
    remove redundant propositions in the clause
    '''
    pos = clause[0]
    neg = clause[1]

    pos = list(set(pos))
    neg = list(set(neg))

    return [pos, neg]

In [4]:
def resolve(clause1: list[list[str]], clause2: list[list[str]]) -> list[list[str]]:
    '''
    function: resolve()

    [['p', 'q'] ['r']] : clause P ∨ Q ∨ ¬R

    resolve clauses c1 and c2
    - if resolvable returns resolvent
    - if not resolvable returns False
    - returns empty clause [[], []] when Falsified

    '''

    # remove any redundant propositions in the clauses
    c1 = mk_unique(clause1)
    c2 = mk_unique(clause2)

    count = 0

    # print clauses
    if DEBUG:
        print(f"resolving: {to_string(c1)} and {to_string(c2)}")

    # c1 pos vs. c2 neg
    for p1 in c1[0]:
        for p2 in c2[1]:
            if p1 == p2:
                c1 = [rm_item(c1[0], p1), c1[1]]
                c2 = [c2[0], rm_item(c2[1], p2)]
                count = count+1

    # c2 pos vs. c1 neg
    for p1 in c1[1]:
        for p2 in c2[0]:
            if p1 == p2:
                c1 = [c1[0], rm_item(c1[1], p1)]
                c2 = [rm_item(c2[0], p2), c2[1]]
                count = count+1

    # check for multiple matches or no match and abort if so.
    if count > 1 or count == 0:
        return False

    # compute resolvent
    c1[0].extend(c2[0])
    c1[1].extend(c2[1])

    # make unique and return

    return mk_unique(c1)

In [5]:
def null_p(clause):
    '''
    function null_p()

    check if "False" is derived
    - returns True if empty clause
    - returns False if non-empty clause
    '''

    if (type(clause) is list) and len(clause[0]) + len(clause[1]) == 0:
        return True
    else:
        return False

In [6]:
def to_string(clause):
    '''
    function print_clause

    See function resolve() for clause representation.
    '''

    s = ""
    for pos in clause[0]:
        s += f"{pos} \u2228 "

    for neg in clause[1]:
        s += f"\u00AC{neg} \u2228 "

    if not len(s):
        return "NULL"
    n = len(s)
    return s[0:(n-2)]

In [7]:
def print_clause(clause):
    '''
    function print_clause

    See function resolve() for clause representation.

    '''
    print(to_string(clause))

Using the `resolve()` function and the null_p function, implement the two-pointer method for propositional logic theorem proving
- use set of support

Theorem representation:  
`[ [<index> <clause1>], [<index> <clause2>], ... ]`

index: integer

clause representation:  
`[<positive-proposition-list>, <negative-proposition-list>]`

proposition-list representation:  
`[ 'p', 'q', 'r' ]`

In [8]:
# define your theorem, as a set of indexed clauses
theorem = [    # pos-list      # neg-list
           [0, [['p','q'],    ['r']]],       # clause 0     P \/ Q \/ ~R
           [1, [['s'],        ['q']]],       # clause 1     S \/ ~Q
           [2, [['t'],        ['s']]],       # clause 2     T \/ ~S
           [3, [[],           ['p']]],       # clause 3     ~P   - negated conclusion starts here (goal clause index = 3)
           [4, [['r'],        []]],          # clause 4     R
           [5, [[],           ['t']]]        # clause 5     ~T
          ]

# Homework Theorem
theorem2 = [
            [0, [['I'],         ['M']]],    # I ∨ ¬M
            [1, [['M'],         ['I']]],    # M ∨ ¬I
            [2, [['M', 'L'],    []]],       # M ∨ L
            [3, [['H'],         ['I']]],    # H ∨ ¬I
            [4, [['H'],         ['L']]],    # H ∨ ¬L
            [5, [['G'],         ['H']]],    # H ∨ ¬G
            [6, [[],            ['M']]]     # ¬M    - negated conclusion starts here (goal clause index = 6)
           ]

# Theorem from the Exam
theorem3 = [
            [0, [['s', 'q', 'w'],   []]],           # S ∨ Q ∨ W
            [1, [['w'],             ['r']]],        # W ∨ ¬R
            [2, [[],                ['p', 's']]],   # ¬P ∨ ¬S
            [3, [['r'],             ['q']]],        # R ∨ ¬Q
            [4, [['p'],             []]],           # P         - negated conclusion starts here (goal clause index = 4)
            [5, [[],                ['w']]],        # ¬W
           ]

In [9]:
def prover(thm, goal):
    '''
    function prover: implement this

    two-pointer method, with set of support
    - arguments:
        thm : theorem
        goal : integer index (clause number where the negated conclusion starts)
    - show resolution steps
    - if null_p checks, return True (theorem proven)
    - otherwise return False
    '''

    print('\nprover():\n\nTheorem:')

    for num, clause in thm:
        print(f"clause {num}: {to_string(clause)}")

    print(f"Goal clause index = {goal}\n")

    resolved_clauses = [clause for _, clause in thm]

    # Continue resolving clauses until no further resolutions are possible
    while True:
        new_resolved_clauses = []  # To store newly resolved clauses in this iteration

        for i, clause_i in enumerate(resolved_clauses):
            for clause_j in resolved_clauses[i + 1:]:
                resolvent = resolve(clause_i, clause_j)

                # Check if the resolvent is a null clause
                if null_p(resolvent):
                    print("Null clause found. The theorem is proven.")
                    return True
                # Check if resolving failed
                if not resolvent:
                    print("Cannot resolve")
                    continue
                # Check if resolvent is already in resolved_clauses
                if resolvent in resolved_clauses:
                    print("Redundent resolve")
                    continue
                print(f"Resolved: {to_string(resolvent)}")
                new_resolved_clauses.append(resolvent)
            print("")


        # Check if no further resolutions are possible
        if not new_resolved_clauses:
            print("No further resolutions possible. The theorem cannot be proven.")
            return False

        resolved_clauses.extend(new_resolved_clauses)

    return True


In [10]:
prover(theorem, 3)


prover():

Theorem:
clause 0: p ∨ q ∨ ¬r 
clause 1: s ∨ ¬q 
clause 2: t ∨ ¬s 
clause 3: ¬p 
clause 4: r 
clause 5: ¬t 
Goal clause index = 3

resolving: p ∨ q ∨ ¬r  and s ∨ ¬q 
Resolved: p ∨ s ∨ ¬r 
resolving: p ∨ q ∨ ¬r  and t ∨ ¬s 
Cannot resolve
resolving: p ∨ q ∨ ¬r  and ¬p 
Resolved: q ∨ ¬r 
resolving: p ∨ q ∨ ¬r  and r 
Resolved: p ∨ q 
resolving: p ∨ q ∨ ¬r  and ¬t 
Cannot resolve

resolving: s ∨ ¬q  and t ∨ ¬s 
Resolved: t ∨ ¬q 
resolving: s ∨ ¬q  and ¬p 
Cannot resolve
resolving: s ∨ ¬q  and r 
Cannot resolve
resolving: s ∨ ¬q  and ¬t 
Cannot resolve

resolving: t ∨ ¬s  and ¬p 
Cannot resolve
resolving: t ∨ ¬s  and r 
Cannot resolve
resolving: t ∨ ¬s  and ¬t 
Resolved: ¬s 

resolving: ¬p  and r 
Cannot resolve
resolving: ¬p  and ¬t 
Cannot resolve

resolving: r  and ¬t 
Cannot resolve


resolving: p ∨ q ∨ ¬r  and s ∨ ¬q 
Redundent resolve
resolving: p ∨ q ∨ ¬r  and t ∨ ¬s 
Cannot resolve
resolving: p ∨ q ∨ ¬r  and ¬p 
Redundent resolve
resolving: p ∨ q ∨ ¬r  and r 
Redundent 

True

In [11]:
prover(theorem2, 6)


prover():

Theorem:
clause 0: I ∨ ¬M 
clause 1: M ∨ ¬I 
clause 2: M ∨ L 
clause 3: H ∨ ¬I 
clause 4: H ∨ ¬L 
clause 5: G ∨ ¬H 
clause 6: ¬M 
Goal clause index = 6

resolving: I ∨ ¬M  and M ∨ ¬I 
Cannot resolve
resolving: I ∨ ¬M  and L ∨ M 
Resolved: L ∨ I 
resolving: I ∨ ¬M  and H ∨ ¬I 
Resolved: H ∨ ¬M 
resolving: I ∨ ¬M  and H ∨ ¬L 
Cannot resolve
resolving: I ∨ ¬M  and G ∨ ¬H 
Cannot resolve
resolving: I ∨ ¬M  and ¬M 
Cannot resolve

resolving: M ∨ ¬I  and L ∨ M 
Cannot resolve
resolving: M ∨ ¬I  and H ∨ ¬I 
Cannot resolve
resolving: M ∨ ¬I  and H ∨ ¬L 
Cannot resolve
resolving: M ∨ ¬I  and G ∨ ¬H 
Cannot resolve
resolving: M ∨ ¬I  and ¬M 
Resolved: ¬I 

resolving: L ∨ M  and H ∨ ¬I 
Cannot resolve
resolving: L ∨ M  and H ∨ ¬L 
Resolved: H ∨ M 
resolving: L ∨ M  and G ∨ ¬H 
Cannot resolve
resolving: L ∨ M  and ¬M 
Resolved: L 

resolving: H ∨ ¬I  and H ∨ ¬L 
Cannot resolve
resolving: H ∨ ¬I  and G ∨ ¬H 
Resolved: G ∨ ¬I 
resolving: H ∨ ¬I  and ¬M 
Cannot resolve

resolving: H ∨ ¬L 

False

In [12]:
prover(theorem3, 4)


prover():

Theorem:
clause 0: s ∨ q ∨ w 
clause 1: w ∨ ¬r 
clause 2: ¬p ∨ ¬s 
clause 3: r ∨ ¬q 
clause 4: p 
clause 5: ¬w 
Goal clause index = 4

resolving: s ∨ q ∨ w  and w ∨ ¬r 
Cannot resolve
resolving: s ∨ q ∨ w  and ¬p ∨ ¬s 
Resolved: q ∨ w ∨ ¬p 
resolving: s ∨ q ∨ w  and r ∨ ¬q 
Resolved: s ∨ r ∨ w 
resolving: s ∨ q ∨ w  and p 
Cannot resolve
resolving: s ∨ q ∨ w  and ¬w 
Resolved: s ∨ q 

resolving: w ∨ ¬r  and ¬p ∨ ¬s 
Cannot resolve
resolving: w ∨ ¬r  and r ∨ ¬q 
Resolved: w ∨ ¬q 
resolving: w ∨ ¬r  and p 
Cannot resolve
resolving: w ∨ ¬r  and ¬w 
Resolved: ¬r 

resolving: ¬p ∨ ¬s  and r ∨ ¬q 
Cannot resolve
resolving: ¬p ∨ ¬s  and p 
Resolved: ¬s 
resolving: ¬p ∨ ¬s  and ¬w 
Cannot resolve

resolving: r ∨ ¬q  and p 
Cannot resolve
resolving: r ∨ ¬q  and ¬w 
Cannot resolve

resolving: p  and ¬w 
Cannot resolve


resolving: s ∨ q ∨ w  and w ∨ ¬r 
Cannot resolve
resolving: s ∨ q ∨ w  and ¬p ∨ ¬s 
Redundent resolve
resolving: s ∨ q ∨ w  and r ∨ ¬q 
Redundent resolve
resolving: s

True